In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, scale
from sklearn.metrics import roc_auc_score

In [2]:
train=pd.read_csv('train.csv')
#test=pd.read_csv('test.csv')

In [41]:
features = [c for c in train.columns if c not in ['ID_code', 'target']]

In [42]:
target = train['target']

In [3]:
y=train['target']
X = train.drop(['target', 'ID_code'], axis=1)

KeyError: 'target'

In [31]:
ID_code=test['ID_code']
X_test = test.drop(['ID_code'],axis = 1)

In [53]:
sc = StandardScaler()
X= sc.fit_transform(X)
X_test= sc.fit_transform(X_test)

In [32]:
X.shape, y.shape, X_test.shape

((200000, 200), (200000,), (200000, 200))

In [14]:
n_splits = 5 # Number of K-fold Splits
splits = list(StratifiedKFold(n_splits=n_splits, shuffle=True).split(X, y))

In [44]:
from sklearn.model_selection import KFold
splits = list(KFold(n_splits=2, shuffle=True).split(X, y))

In [43]:
n_splits = 15
splits = StratifiedKFold(n_splits=kfold, shuffle=False, random_state=1111)

In [18]:
X.shape,y.shape

((200000, 200), (200000,))

In [34]:
splits[0][0].shape

(186666,)

## LightGBM

In [5]:
import lightgbm as lgb

In [24]:
lgb_param = {
    # This one seems to be better than the lower one
    'bagging_freq': 5,
        'bagging_fraction': 0.38,
        'boost_from_average':'false',
        'boost': 'gbdt',
        'feature_fraction': 0.045,
        'learning_rate': 0.01,
        'max_depth': -1,  
        'metric':'auc',
        'min_data_in_leaf': 80,
        'min_sum_hessian_in_leaf': 10.0,
        'num_leaves': 13,
        'num_threads': 8,
        'tree_learner': 'serial',
        'objective': 'binary', 
        'verbosity': 1
}

In [9]:
lgb_param = {
        'num_leaves': 9,
        'max_bin': 63,
        'min_data_in_leaf': 45,
        'learning_rate': 0.01,
        'min_sum_hessian_in_leaf': 0.000446,
        'bagging_fraction': 0.55, 
        'bagging_freq': 5, 
        'max_depth': 15,
        'save_binary': True,
        'seed': 42,
        'feature_fraction_seed': 31415,
         'feature_fraction': 0.04,
        'bagging_seed': 42,
        'drop_seed': 42,
        'data_random_seed': 42,
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': 1,
        'metric': 'auc',
        'is_unbalance': True,
        'boost_from_average': False,
    }

In [46]:
oof = np.zeros(len(train))
predictions = np.zeros(len(test))
for fold_, (trn_idx, val_idx) in enumerate(splits.split(train.values, target.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train.iloc[val_idx][features], label=target.iloc[val_idx])
    num_round=1000000
    lgb_clf = lgb.train(lgb_param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 3500)
    oof[val_idx] = lgb_clf.predict(train.iloc[val_idx][features], num_iteration=lgb_clf.best_iteration)
    predictions += lgb_clf.predict(test[features], num_iteration=lgb_clf.best_iteration) / folds.n_splits
print("CV score: {:<8.5f}".format(roc_auc_score(target.values, oof)))

Fold 0
Training until validation scores don't improve for 3500 rounds.
[1000]	training's auc: 0.900094	valid_1's auc: 0.882715
[2000]	training's auc: 0.911258	valid_1's auc: 0.891459
[3000]	training's auc: 0.918328	valid_1's auc: 0.89572
[4000]	training's auc: 0.923701	valid_1's auc: 0.898419
[5000]	training's auc: 0.928324	valid_1's auc: 0.899843
[6000]	training's auc: 0.932349	valid_1's auc: 0.900891
[7000]	training's auc: 0.936091	valid_1's auc: 0.901432
[8000]	training's auc: 0.939684	valid_1's auc: 0.901907
[9000]	training's auc: 0.943039	valid_1's auc: 0.902295
[10000]	training's auc: 0.946273	valid_1's auc: 0.90254
[11000]	training's auc: 0.949378	valid_1's auc: 0.902608
[12000]	training's auc: 0.952431	valid_1's auc: 0.902609
[13000]	training's auc: 0.95527	valid_1's auc: 0.902509
[14000]	training's auc: 0.958014	valid_1's auc: 0.902471
Early stopping, best iteration is:
[11185]	training's auc: 0.94996	valid_1's auc: 0.902728
Fold 1
Training until validation scores don't improv

[14000]	training's auc: 0.95801	valid_1's auc: 0.903903
Early stopping, best iteration is:
[10517]	training's auc: 0.947833	valid_1's auc: 0.904756
Fold 9
Training until validation scores don't improve for 3500 rounds.
[1000]	training's auc: 0.900211	valid_1's auc: 0.888288
[2000]	training's auc: 0.911071	valid_1's auc: 0.895479
[3000]	training's auc: 0.918254	valid_1's auc: 0.899136
[4000]	training's auc: 0.923711	valid_1's auc: 0.90089
[5000]	training's auc: 0.928323	valid_1's auc: 0.901566
[6000]	training's auc: 0.93245	valid_1's auc: 0.902072
[7000]	training's auc: 0.936243	valid_1's auc: 0.902036
[8000]	training's auc: 0.939754	valid_1's auc: 0.90201
[9000]	training's auc: 0.943117	valid_1's auc: 0.902093
[10000]	training's auc: 0.946323	valid_1's auc: 0.902213
[11000]	training's auc: 0.949436	valid_1's auc: 0.902271
[12000]	training's auc: 0.952407	valid_1's auc: 0.902263
[13000]	training's auc: 0.955256	valid_1's auc: 0.902065
[14000]	training's auc: 0.958042	valid_1's auc: 0.90

In [25]:
def lgb_clf_train(train_idx, valid_idx):
    trn_data = lgb.Dataset(X[train_idx.astype(int)], label=y[train_idx.astype(int)])
    val_data = lgb.Dataset(X[valid_idx.astype(int)], label=y[valid_idx.astype(int)])   
    num_round = 1000000
    lgb_clf = lgb.train(lgb_param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 3500)
    return lgb_clf

## XGboost

In [11]:
import xgboost as xgb

In [12]:
xgb_param = {
    'min_child_weight': 10.0,
    'objective': 'binary:logistic',
    'max_depth': 7,
    'max_delta_step': 1.8,
    'colsample_bytree': 0.4,
    'subsample': 0.8,
    'eta': 0.025,
    'gamma': 0.65,
    'eval_metric':'auc'
        }

In [13]:
def xgb_clf_train(train_idx, valid_idx):
    trn_data = xgb.DMatrix(X[train_idx.astype(int)], label=y[train_idx.astype(int)])
    val_data = xgb.DMatrix(X[valid_idx.astype(int)], label=y[valid_idx.astype(int)])
    watchlist = [(trn_data, 'train'), (val_data, 'valid')]
    xgb_clf=xgb.train(xgb_param, trn_data, 700, evals=watchlist, early_stopping_rounds=250, verbose_eval=500)
    return xgb_clf

## DNN

In [14]:
import tensorflow as tf

In [15]:
he_init = tf.variance_scaling_initializer()

In [16]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import NotFittedError

class DNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden_layers=5, n_neurons=100, optimizer_class=tf.train.AdamOptimizer,
                 learning_rate=0.01, batch_size=20, activation=tf.nn.elu, initializer=he_init,
                 batch_norm_momentum=None, dropout_rate=None, random_state=None):
        """Initialize the DNNClassifier by simply storing all the hyperparameters."""
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        self.initializer = initializer
        self.batch_norm_momentum = batch_norm_momentum
        self.dropout_rate = dropout_rate
        self.random_state = random_state
        self._session = None

    def _dnn(self, inputs):
        """Build the hidden layers, with support for batch normalization and dropout."""
        for layer in range(self.n_hidden_layers):
            if self.dropout_rate:
                inputs = tf.layers.dropout(inputs, self.dropout_rate, training=self._training)
            inputs = tf.layers.dense(inputs, self.n_neurons,
                                     kernel_initializer=self.initializer,
                                     name="hidden%d" % (layer + 1))
            if self.batch_norm_momentum:
                inputs = tf.layers.batch_normalization(inputs, momentum=self.batch_norm_momentum,
                                                       training=self._training)
            inputs = self.activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs

    def _build_graph(self, n_inputs, n_outputs):
        """Build the same model as earlier"""
        if self.random_state is not None:
            tf.set_random_seed(self.random_state)
            np.random.seed(self.random_state)

        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
        y = tf.placeholder(tf.int32, shape=(None), name="y")

        if self.batch_norm_momentum or self.dropout_rate:
            self._training = tf.placeholder_with_default(False, shape=(), name='training')
        else:
            self._training = None

        dnn_outputs = self._dnn(X)

        logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
        Y_proba = tf.nn.softmax(logits, name="Y_proba")

        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                                  logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")

        optimizer = self.optimizer_class(learning_rate=self.learning_rate)
        training_op = optimizer.minimize(loss)

        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()

        # Make the important operations available easily through instance variables
        self._X, self._y = X, y
        self._Y_proba, self._loss = Y_proba, loss
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver

    def close_session(self):
        if self._session:
            self._session.close()

    def _get_model_params(self):
        """Get all variable values (used for early stopping, faster than saving to disk)"""
        with self._graph.as_default():
            gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        return {gvar.op.name: value for gvar, value in zip(gvars, self._session.run(gvars))}

    def _restore_model_params(self, model_params):
        """Set all variables to the given values (for early stopping, faster than loading from disk)"""
        gvar_names = list(model_params.keys())
        assign_ops = {gvar_name: self._graph.get_operation_by_name(gvar_name + "/Assign")
                      for gvar_name in gvar_names}
        init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
        feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
        self._session.run(assign_ops, feed_dict=feed_dict)

    def fit(self, X, y, n_epochs=100, X_valid=None, y_valid=None):
        """Fit the model to the training set. If X_valid and y_valid are provided, use early stopping."""
        self.close_session()

        # infer n_inputs and n_outputs from the training set.
        n_inputs = X.shape[1]
        self.classes_ = np.unique(y)
        n_outputs = len(self.classes_)
        # Translate the labels vector to a vector of sorted class indices, containing
        # integers from 0 to n_outputs - 1.
        # For example, if y is equal to [8, 8, 9, 5, 7, 6, 6, 6], then the sorted class
        # labels (self.classes_) will be equal to [5, 6, 7, 8, 9], and the labels vector
        # will be translated to [3, 3, 4, 0, 2, 1, 1, 1]
        self.class_to_index_ = {label: index
                                for index, label in enumerate(self.classes_)}
        y = np.array([self.class_to_index_[label]
                      for label in y], dtype=np.int32)
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(n_inputs, n_outputs)
            # extra ops for batch normalization
            extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

        # needed in case of early stopping
        max_checks_without_progress = 20
        checks_without_progress = 0
        best_loss = np.infty
        best_params = None
        
        # Now train the model!
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._init.run()
            for epoch in range(n_epochs):
                start_pos=0
                for batches in range(int(len(X)/self.batch_size)):
                    X_batch,y_batch=X[start_pos:start_pos+self.batch_size],y[start_pos:start_pos+self.batch_size]
                    feed_dict = {self._X: X_batch, self._y: y_batch}
                    start_pos+=self.batch_size
                #rnd_idx = np.random.permutation(len(X))
                #for rnd_indices in np.array_split(rnd_idx, len(X) // self.batch_size):
                    #X_batch, y_batch = X[rnd_indices], y[rnd_indices]
                    #feed_dict = {self._X: X_batch, self._y: y_batch}
                    if self._training is not None:
                        feed_dict[self._training] = True
                    sess.run(self._training_op, feed_dict=feed_dict)
                    if extra_update_ops:
                        sess.run(extra_update_ops, feed_dict=feed_dict)
                if X_valid is not None and y_valid is not None:
                    loss_val, acc_val = sess.run([self._loss, self._accuracy],
                                                 feed_dict={self._X: X_valid,
                                                            self._y: y_valid})
                    if loss_val < best_loss:
                        best_params = self._get_model_params()
                        best_loss = loss_val
                        checks_without_progress = 0
                    else:
                        checks_without_progress += 1
                    print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_val, best_loss, acc_val * 100))
                    if checks_without_progress > max_checks_without_progress:
                        print("Early stopping!")
                        break
                else:
                    loss_train, acc_train = sess.run([self._loss, self._accuracy],
                                                     feed_dict={self._X: X_batch,
                                                                self._y: y_batch})
                    print("{}\tLast training batch loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_train, acc_train * 100))
            # If we used early stopping then rollback to the best model found
            if best_params:
                self._restore_model_params(best_params)
            return self

    def predict_proba(self, X):
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        with self._session.as_default() as sess:
            return self._Y_proba.eval(feed_dict={self._X: X})

    def predict(self, X):
        class_indices = np.argmax(self.predict_proba(X), axis=1)
        return np.array([[self.classes_[class_index]]
                         for class_index in class_indices], np.int32)

    def save(self, path):
        self._saver.save(self._session, path)

In [17]:
def leaky_relu(alpha=0.01):
    def parametrized_leaky_relu(z, name=None):
        return tf.maximum(alpha * z, z, name=name)
    return parametrized_leaky_relu

In [18]:
def dnn_clf_train(train_idx, valid_idx):
    X_train=X[train_idx.astype(int)]
    y_train=y[train_idx.astype(int)]
    X_valid=X[valid_idx.astype(int)]
    y_valid=y[valid_idx.astype(int)]
    
    dnn_clf = DNNClassifier(random_state=42, n_neurons=10,n_hidden_layers=5,learning_rate=0.01, 
                            batch_size=100,activation=leaky_relu(alpha=0.01))
    dnn_clf.fit(X_train, y_train, n_epochs=1000, X_valid=X_valid, y_valid=y_valid)
    return dnn_clf

## Ensemble stacking

In [19]:
from sklearn.ensemble import RandomForestClassifier

D:\Anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [38]:
predictions = np.zeros(len(X_test))
oof=np.zeros(len(X))

In [21]:
for i, (train_idx, valid_idx) in enumerate(splits):  
    X_train_predictions = np.empty((160000, 3), dtype=np.float32)
    print(f'Fold {i + 1}')
    #lightBGM training
    lgb_clf=lgb_clf_train(train_idx, valid_idx)
    X_train_predictions[:, 0] = lgb_clf.predict(X[train_idx.astype(int)])
    
    #XGboost
    xgb_clf=xgb_clf_train(train_idx, valid_idx)
    xgb_train=xgb.DMatrix(X[train_idx.astype(int)])
    X_train_predictions[:, 1] = xgb_clf.predict(xgb_train)
    
    #DNN
    dnn_clf = dnn_clf_train(train_idx, valid_idx)
    _,X_train_predictions[:, 2] = zip(*dnn_clf.predict_proba(X[train_idx.astype(int)]))
    
    #Stack with RF
    rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
    rnd_forest_blender.fit(X_train_predictions, y[train_idx.astype(int)])
    
    
    X_valid_predictions = np.empty((40000, 3), dtype=np.float32)
    X_valid_predictions[:, 0] = lgb_clf.predict(X[valid_idx.astype(int)])
    xgb_valid=xgb.DMatrix(X[valid_idx.astype(int)])
    X_valid_predictions[:, 1] = xgb_clf.predict(xgb_valid)
    _,X_valid_predictions[:, 2] = zip(*dnn_clf.predict_proba(X[valid_idx.astype(int)]))
    _,oof[valid_idx] =  zip(*rnd_forest_blender.predict_proba(X_valid_predictions))
    
    # Prediction
    X_test_predictions = np.empty((len(X_test), 3), dtype=np.float32)
    X_test_predictions[:, 0] = lgb_clf.predict(X_test)
    xgb_test=xgb.DMatrix(X_test)
    X_test_predictions[:, 1] = xgb_clf.predict(xgb_test)
    _, X_test_predictions[:, 2] = zip(*dnn_clf.predict_proba(X_test))
    
    l1, l2=zip(*rnd_forest_blender.predict_proba(X_test_predictions))
    predictions += np.array(l2)/5

print("CV score: {:<8.5f}".format(roc_auc_score(y, oof)))

Fold 1
Training until validation scores don't improve for 250 rounds.
[1000]	training's auc: 0.89639	valid_1's auc: 0.883432
[2000]	training's auc: 0.904721	valid_1's auc: 0.888555
[3000]	training's auc: 0.91086	valid_1's auc: 0.892143
[4000]	training's auc: 0.915808	valid_1's auc: 0.894727
[5000]	training's auc: 0.920147	valid_1's auc: 0.896338
[6000]	training's auc: 0.924023	valid_1's auc: 0.897533
[7000]	training's auc: 0.927569	valid_1's auc: 0.898356
[8000]	training's auc: 0.930817	valid_1's auc: 0.898801
[9000]	training's auc: 0.934048	valid_1's auc: 0.899181
Early stopping, best iteration is:
[9686]	training's auc: 0.936126	valid_1's auc: 0.899348
[01:23:28] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 142 extra nodes, 2 pruned nodes, max_depth=7
[0]	train-auc:0.632309	valid-auc:0.619916
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 250 rounds.

[01:24:14] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 132 extra nodes, 0 pruned nodes, max_depth=7
[01:24:15] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 152 extra nodes, 0 pruned nodes, max_depth=7
[01:24:15] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 150 extra nodes, 0 pruned nodes, max_depth=7
[01:24:16] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 168 extra nodes, 0 pruned nodes, max_depth=7
[01:24:17] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 144 extra nodes, 0 pruned nodes, max_depth=7
[01:24:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 136 extra nodes, 0 pruned nodes, max_depth=7
[01:24:19] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pru

[01:25:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 158 extra nodes, 0 pruned nodes, max_depth=7
[01:25:07] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 156 extra nodes, 0 pruned nodes, max_depth=7
[01:25:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 142 extra nodes, 0 pruned nodes, max_depth=7
[01:25:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 184 extra nodes, 0 pruned nodes, max_depth=7
[01:25:10] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 174 extra nodes, 0 pruned nodes, max_depth=7
[01:25:11] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 156 extra nodes, 2 pruned nodes, max_depth=7
[01:25:12] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pru

[01:25:59] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 154 extra nodes, 0 pruned nodes, max_depth=7
[01:26:00] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 198 extra nodes, 0 pruned nodes, max_depth=7
[01:26:01] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 142 extra nodes, 0 pruned nodes, max_depth=7
[01:26:02] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 158 extra nodes, 0 pruned nodes, max_depth=7
[01:26:03] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 146 extra nodes, 0 pruned nodes, max_depth=7
[01:26:04] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 150 extra nodes, 0 pruned nodes, max_depth=7
[01:26:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pru

[01:26:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 140 extra nodes, 0 pruned nodes, max_depth=7
[01:26:54] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 164 extra nodes, 0 pruned nodes, max_depth=7
[01:26:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 154 extra nodes, 0 pruned nodes, max_depth=7
[01:26:56] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 140 extra nodes, 0 pruned nodes, max_depth=7
[01:26:57] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 134 extra nodes, 0 pruned nodes, max_depth=7
[01:26:58] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 154 extra nodes, 0 pruned nodes, max_depth=7
[01:26:59] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pru

...

[02:42:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 136 extra nodes, 0 pruned nodes, max_depth=7
[02:42:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 128 extra nodes, 0 pruned nodes, max_depth=7
[02:42:53] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 150 extra nodes, 0 pruned nodes, max_depth=7
[02:42:54] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 142 extra nodes, 0 pruned nodes, max_depth=7
[02:42:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=7
[02:42:56] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 130 extra nodes, 0 pruned nodes, max_depth=7
[02:42:57] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pru

[02:43:45] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 182 extra nodes, 0 pruned nodes, max_depth=7
[02:43:46] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 158 extra nodes, 0 pruned nodes, max_depth=7
[02:43:47] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 154 extra nodes, 0 pruned nodes, max_depth=7
[02:43:49] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 148 extra nodes, 0 pruned nodes, max_depth=7
[02:43:50] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 162 extra nodes, 0 pruned nodes, max_depth=7
[02:43:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 132 extra nodes, 0 pruned nodes, max_depth=7
[02:43:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pru

[02:44:39] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 148 extra nodes, 0 pruned nodes, max_depth=7
[02:44:40] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 160 extra nodes, 0 pruned nodes, max_depth=7
[02:44:41] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 130 extra nodes, 0 pruned nodes, max_depth=7
[02:44:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_depth=7
[02:44:43] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 134 extra nodes, 0 pruned nodes, max_depth=7
[02:44:44] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 142 extra nodes, 0 pruned nodes, max_depth=7
[02:44:45] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pru

[02:45:33] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 170 extra nodes, 0 pruned nodes, max_depth=7
[02:45:34] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 160 extra nodes, 0 pruned nodes, max_depth=7
[02:45:35] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 158 extra nodes, 0 pruned nodes, max_depth=7
[02:45:36] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 160 extra nodes, 0 pruned nodes, max_depth=7
[02:45:37] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 186 extra nodes, 0 pruned nodes, max_depth=7
[02:45:38] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=7
[02:45:39] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pru

[02:46:27] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 132 extra nodes, 0 pruned nodes, max_depth=7
[02:46:28] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 130 extra nodes, 0 pruned nodes, max_depth=7
[02:46:29] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 168 extra nodes, 0 pruned nodes, max_depth=7
[02:46:30] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=7
[02:46:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 164 extra nodes, 0 pruned nodes, max_depth=7
[02:46:32] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 138 extra nodes, 0 pruned nodes, max_depth=7
[02:46:33] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pru

In [47]:
submission=pd.DataFrame()
submission['ID_code']=ID_code
submission['target']=predictions
submission.to_csv('sub_lgb_v3.csv', index=False)

In [37]:
oof=np.zeros(len(X))
lgb_pred=np.zeros(len(X_test))
for i, (train_idx, valid_idx) in enumerate(splits.split(X.values, y.values)):  
    #X_train_predictions = np.empty((40000, 3), dtype=np.float32)
    print(f'Fold {i + 1}')
    #lightBGM training
    lgb_clf=lgb_clf_train(train_idx, valid_idx)
    #X_train_predictions[:, 0] = lgb_clf.predict(X[train_idx.astype(int)],num_iteration=lgb_clf.best_iteration)
    oof[valid_idx]=lgb_clf.predict(X[valid_idx])
    lgb_pred+=lgb_clf.predict(X_test)/15
print("CV score: {:<8.5f}".format(roc_auc_score(y, oof)))    

Fold 1


KeyError: '[ 13305  13306  13308 ... 199997 199998 199999] not in index'

In [30]:
oof=np.zeros(len(X))
xgb_pred=np.zeros(len(X_test))
for i, (train_idx, valid_idx) in enumerate(splits):  
    print(f'Fold {i + 1}')
    #xgBoosttraining
    #xgb_clf=lgb_clf_train(train_idx, valid_idx)
    #xgb_train=xgb.DMatrix(X[train_idx.astype(int)])
    #X_train_predictions[:, 1] = xgb_clf.predict(X[train_idx.astype(int)])
    xgb_valid=xgb.DMatrix(X[valid_idx])
    oof[valid_idx]=xgb_clf.predict(xgb_valid)
    xgb_test=xgb.DMatrix(X_test)
    xgb_pred+=xgb_clf.predict(xgb_test)/5
print("CV score: {:<8.5f}".format(roc_auc_score(y, oof))) 

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
CV score: 0.95966 


In [35]:
oof=np.zeros(len(X))
dnn_pred=np.zeros(len(X_test))
for i, (train_idx, valid_idx) in enumerate(splits):  
    print(f'Fold {i + 1}')
    #dnn_clf = dnn_clf_train(train_idx, valid_idx)
    #_, X_train_predictions[:, 2] = zip(*dnn_clf.predict_proba(X[train_idx]))
    _, oof[valid_idx]=zip(*dnn_clf.predict_proba(X[valid_idx]))
    l1, l2=zip(*dnn_clf.predict_proba(X_test))
    dnn_pred+=np.array(l2)/5
print("CV score: {:<8.5f}".format(roc_auc_score(y, oof))) 

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
CV score: 0.86460 


In [27]:
dnn_clf._get_model_params

<bound method DNNClassifier._get_model_params of DNNClassifier(activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x000001CB04548BF8>,
       batch_norm_momentum=None, batch_size=100, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x000001CB1C421C50>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=10,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)>

In [59]:
rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_train_predictions, y[train_idx.astype(int)])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=True, random_state=42, verbose=0, warm_start=False)

In [61]:
result=rnd_forest_blender.predict_proba(X_train_predictions)

In [77]:
np.mean(np.array(l2)/3)

0.027741741666666667

In [47]:
from sklearn.linear_model import LogisticRegression
for i, (train_idx, valid_idx) in enumerate(splits):  
    X_train_predictions = np.empty((100000, 3), dtype=np.float32)
    print(f'Fold {i + 1}')
    #lightBGM training
    lgb_clf=lgb_clf_train(train_idx, valid_idx)
    X_train_predictions[:, 0] = lgb_clf.predict(X[train_idx.astype(int)])
    
    #XGboost
    xgb_clf=xgb_clf_train(train_idx, valid_idx)
    xgb_train=xgb.DMatrix(X[train_idx.astype(int)])
    X_train_predictions[:, 1] = xgb_clf.predict(xgb_train)
    
    #DNN
    dnn_clf = dnn_clf_train(train_idx, valid_idx)
    _,X_train_predictions[:, 2] = zip(*dnn_clf.predict_proba(X[train_idx.astype(int)]))
    
    #Stack with LR
    
    logistic_blender=LogisticRegression()
    logistic_blender.fit(X_train_predictions, y[train_idx.astype(int)])
    
    #Valid sets prediction
    X_valid_predictions = np.empty((100000, 3), dtype=np.float32)
    X_valid_predictions[:, 0] = lgb_clf.predict(X[valid_idx.astype(int)])
    xgb_valid=xgb.DMatrix(X[valid_idx.astype(int)])
    X_valid_predictions[:, 1] = xgb_clf.predict(xgb_valid)
    _,X_valid_predictions[:, 2] = zip(*dnn_clf.predict_proba(X[valid_idx.astype(int)]))
    _,oof[valid_idx] =  zip(*logistic_blender.predict_proba(X_valid_predictions))
    
    # Prediction
    X_test_predictions = np.empty((len(X_test), 3), dtype=np.float32)
    X_test_predictions[:, 0] = lgb_clf.predict(X_test)
    xgb_test=xgb.DMatrix(X_test)
    X_test_predictions[:, 1] = xgb_clf.predict(xgb_test)
    _, X_test_predictions[:, 2] = zip(*dnn_clf.predict_proba(X_test))
    
    l1, l2=zip(*logistic_blender.predict_proba(X_test_predictions))
    predictions += np.array(l2)/2

print("CV score: {:<8.5f}".format(roc_auc_score(y, oof)))

Fold 1
Training until validation scores don't improve for 250 rounds.
[1000]	training's auc: 0.904344	valid_1's auc: 0.882782
[2000]	training's auc: 0.914078	valid_1's auc: 0.887345
[3000]	training's auc: 0.92152	valid_1's auc: 0.890437
[4000]	training's auc: 0.927518	valid_1's auc: 0.892477
[5000]	training's auc: 0.932917	valid_1's auc: 0.893791
[6000]	training's auc: 0.937733	valid_1's auc: 0.894572
[7000]	training's auc: 0.942348	valid_1's auc: 0.894918
[8000]	training's auc: 0.946609	valid_1's auc: 0.895128
Early stopping, best iteration is:
[8035]	training's auc: 0.946758	valid_1's auc: 0.895151
[12:15:10] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 146 extra nodes, 0 pruned nodes, max_depth=7
[0]	train-auc:0.650239	valid-auc:0.628254
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 250 rounds.
[12:15:10] C:\Users\Administrator\Desktop\xgboost\src\

[12:15:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 134 extra nodes, 0 pruned nodes, max_depth=7
[12:15:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 150 extra nodes, 0 pruned nodes, max_depth=7
[12:15:43] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=7
[12:15:44] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 140 extra nodes, 0 pruned nodes, max_depth=7
[12:15:44] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 136 extra nodes, 0 pruned nodes, max_depth=7
[12:15:45] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 144 extra nodes, 0 pruned nodes, max_depth=7
[12:15:46] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pru

[12:16:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=7
[12:16:19] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=7
[12:16:19] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=7
[12:16:20] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 146 extra nodes, 0 pruned nodes, max_depth=7
[12:16:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 0 pruned nodes, max_depth=7
[12:16:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 132 extra nodes, 0 pruned nodes, max_depth=7
[12:16:22] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pru

[12:16:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 158 extra nodes, 0 pruned nodes, max_depth=7
[12:16:56] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=7
[12:16:56] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=7
[12:16:57] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 144 extra nodes, 0 pruned nodes, max_depth=7
[12:16:58] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 164 extra nodes, 0 pruned nodes, max_depth=7
[12:16:59] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 160 extra nodes, 0 pruned nodes, max_depth=7
[12:16:59] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pru

[12:17:33] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 148 extra nodes, 0 pruned nodes, max_depth=7
[12:17:34] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 150 extra nodes, 0 pruned nodes, max_depth=7
[12:17:34] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 144 extra nodes, 0 pruned nodes, max_depth=7
[12:17:35] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 146 extra nodes, 0 pruned nodes, max_depth=7
[12:17:36] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 0 pruned nodes, max_depth=7
[12:17:36] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 134 extra nodes, 0 pruned nodes, max_depth=7
[12:17:37] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree prun

...

[12:33:10] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=7
[12:33:10] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 134 extra nodes, 0 pruned nodes, max_depth=7
[12:33:11] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=7
[12:33:12] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=7
[12:33:12] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 158 extra nodes, 0 pruned nodes, max_depth=7
[12:33:13] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_depth=7
[12:33:14] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pru

[12:33:47] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 140 extra nodes, 0 pruned nodes, max_depth=7
[12:33:48] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 156 extra nodes, 0 pruned nodes, max_depth=7
[12:33:49] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=7
[12:33:49] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=7
[12:33:50] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 152 extra nodes, 0 pruned nodes, max_depth=7
[12:33:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=7
[12:33:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pru

[12:34:25] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=7
[12:34:25] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 128 extra nodes, 0 pruned nodes, max_depth=7
[12:34:26] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 0 pruned nodes, max_depth=7
[12:34:27] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 132 extra nodes, 0 pruned nodes, max_depth=7
[12:34:27] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 132 extra nodes, 0 pruned nodes, max_depth=7
[12:34:28] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 128 extra nodes, 0 pruned nodes, max_depth=7
[12:34:29] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree prun

[12:35:02] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=7
[12:35:03] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 154 extra nodes, 0 pruned nodes, max_depth=7
[12:35:03] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 0 pruned nodes, max_depth=7
[12:35:04] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=7
[12:35:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 0 pruned nodes, max_depth=7
[12:35:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=7
[12:35:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pru

[12:35:39] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_depth=7
[12:35:40] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=7
[12:35:40] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 156 extra nodes, 0 pruned nodes, max_depth=7
[12:35:41] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=7
[12:35:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 128 extra nodes, 0 pruned nodes, max_depth=7
[12:35:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 100 extra nodes, 0 pruned nodes, max_depth=7
[12:35:43] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pru